# openai playground - function calling

#### Based on Henrik Kniberg - youtube video: [click](https://www.youtube.com/watch?v=i-oHvHejdsc&t=616s)

In [1]:
!pip install openai
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import os
import requests
import json

from dotenv import dotenv_values
from openai import OpenAI

/Users/wojciechnowak/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
secrets = dotenv_values(".env")
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]

client = OpenAI()

## Asking the model without function calling

In [4]:
## Send a request to the model

no_func_shot = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "system", "content": "You give very short answers"},
              {"role": "user", "content": "What is the weather in Stockholm?"}],
)
print(no_func_shot.choices[0].message.content)

I'm sorry, I can't provide current weather updates as I don't have real-time data.


## Asking the model with function calling

![image](func-calling-1.png)

### Prepare function and function spec

In [5]:
# Build weather function spec

weather_function_spec = {
    "name": "get_weather",
    "description": "Get the current temperature & weather for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city",
            },
        },
        "required": ["city"]
    }
}

In [6]:
# Define a function - for a model to be called

def get_weather(city, secrets):
    url = """https://api.openweathermap.org/data/2.5/weather?q={}&appid={}&units=metric""".format(city, secrets["OPEN_WEATHER_API_KEY"])
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("An error has occured, code: {}, reason: {}".format(response.status_code, response.reason))
    data_dict = response.json()

    return json.dumps({
        "city": data_dict['name'],
        "weather": data_dict['weather'][0]['description'],
        "temperature": data_dict['main']['temp'],
        "unit": "Celsius"
    })

### Example 1 - Function Calling

In [7]:
stockholm_weather_msgs = [{"role": "system", "content": "You give very short answers"},
                       {"role": "user", "content": "What is the weather in Stockholm?"}]

shot_1st = client.chat.completions.create(
    model="gpt-4",
    messages=stockholm_weather_msgs,
    functions=[weather_function_spec]
)
llm_response = shot_1st.choices[0].message
llm_response

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "Stockholm"\n}', name='get_weather'), tool_calls=None)

In [8]:
stockholm_weather_msgs.append(llm_response)

args = json.loads(llm_response.function_call.arguments)
weather_response = get_weather(args["city"],secrets)

weather_response

'{"city": "Stockholm", "weather": "clear sky", "temperature": 7.98, "unit": "Celsius"}'

In [9]:
stockholm_weather_msgs.append({"role": "function", "name": "get_weather", "content": weather_response})

shot_2nd  = client.chat.completions.create(
    model="gpt-4",
    messages=stockholm_weather_msgs,
    functions=[weather_function_spec]
)
shot_2nd.choices[0].message

ChatCompletionMessage(content='Clear sky, 7.98°C', refusal=None, role='assistant', function_call=None, tool_calls=None)

### Example 2 - Function Calling

In [10]:
stockholm_paris_weather_msgs = [{"role": "system", "content": "Answer the question about weather."},
                    {"role": "user", "content": "What is the weather sunnier in Stockholm than in Paris?"}]

shot_1st = client.chat.completions.create(
    model="gpt-4",
    messages=stockholm_paris_weather_msgs,
    functions=[weather_function_spec]
)
llm_response = shot_1st.choices[0].message
llm_response

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "Stockholm"\n}', name='get_weather'), tool_calls=None)

In [11]:
stockholm_paris_weather_msgs.append(llm_response)

args = json.loads(llm_response.function_call.arguments)
weather_response = get_weather(args["city"],secrets)

weather_response

'{"city": "Stockholm", "weather": "clear sky", "temperature": 7.98, "unit": "Celsius"}'

In [12]:
stockholm_paris_weather_msgs.append({"role": "function", "name": "get_weather", "content": weather_response})

shot_2nd  = client.chat.completions.create(
    model="gpt-4",
    messages=stockholm_paris_weather_msgs,
    functions=[weather_function_spec]
)
llm_response = shot_2nd.choices[0].message
llm_response

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "Paris"\n}', name='get_weather'), tool_calls=None)

In [13]:
stockholm_paris_weather_msgs.append(llm_response)

args = json.loads(llm_response.function_call.arguments)
weather_response = get_weather(args["city"],secrets)

weather_response

'{"city": "Paris", "weather": "overcast clouds", "temperature": 15.01, "unit": "Celsius"}'

In [14]:
stockholm_paris_weather_msgs.append({"role": "function", "name": "get_weather", "content": weather_response})

shot_3rd  = client.chat.completions.create(
    model="gpt-4",
    messages=stockholm_paris_weather_msgs,
    functions=[weather_function_spec]
)
llm_response = shot_3rd.choices[0].message
llm_response

ChatCompletionMessage(content="Yes, the weather is sunnier in Stockholm than in Paris right now. Stockholm has a clear sky, whereas Paris has overcast clouds. However, Paris is warmer than Stockholm with a temperature of 15.01°C compared to Stockholm's 7.98°C.", refusal=None, role='assistant', function_call=None, tool_calls=None)

### Example 3 - Function Calling

![image](func-calling-2.png)

In [15]:
eu_cities_weather_msgs = [{"role": "system", "content": "Answer the question about weather."},
                    {"role": "user", "content": "Give me the weather in 3 europe largest cities?"}]

In [16]:
# The message for the weather in n europe largest cities would require n function calls!
# Let's use loop to make it more generic

def ask_about_weather(msgs, tool):
    while True:
        shot = client.chat.completions.create(
            model="gpt-4",
            messages=msgs,
            functions=[tool]
        )
    
        response = shot.choices[0].message
        print(response)
        msgs.append(response)

        
        if response.function_call is not None and response.function_call.name == "get_weather":
            args = json.loads(response.function_call.arguments)
            weather_response = get_weather(args["city"], secrets)
            print(weather_response)

            msgs.append({"role": "function", "name": "get_weather", "content": weather_response})
        elif shot.choices[0].finish_reason == "stop":
            return response
    

In [17]:
tool = weather_function_spec

In [18]:
output = ask_about_weather(eu_cities_weather_msgs, tool)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "Moscow"\n}', name='get_weather'), tool_calls=None)
{"city": "Moscow", "weather": "overcast clouds", "temperature": 9.65, "unit": "Celsius"}
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "London"\n}', name='get_weather'), tool_calls=None)
{"city": "London", "weather": "overcast clouds", "temperature": 13.31, "unit": "Celsius"}
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city": "Istanbul"\n}', name='get_weather'), tool_calls=None)
{"city": "Istanbul", "weather": "scattered clouds", "temperature": 14.28, "unit": "Celsius"}
ChatCompletionMessage(content='Here is the current weather in the three largest cities in Europe:\n\n1. Moscow: The weather is overcast clouds with a temperature of 9.65°C\n2. London: The weather is overcast cloud

In [19]:
print(output.content)

Here is the current weather in the three largest cities in Europe:

1. Moscow: The weather is overcast clouds with a temperature of 9.65°C
2. London: The weather is overcast clouds with a temperature of 13.31°C
3. Istanbul: The weather is scattered clouds with a temperature of 14.28°C


In [ ]:
# By adding the loop over a function calling API in we've implemented very simple Agent :)

In [20]:
# The agents have 2 core components:
# 1. Reasoning loop
# 2. Tool abstractions

# Agents have following capabilities:
# 1. Breaking down a complex question into smaller ones
# 2. Choosing an external Tool to use + coming up with parameters for calling the Tool
# 3. Planning out a set of tasks
# 4. Storing previously completed tasks in a memory module

# These capabilities have been demonstrated in this notebook: Example 2 & 3

In [21]:
# Few external links:
# 1. Llama index documentation: https://docs.llamaindex.ai/en/stable/use_cases/agents/
# 2. Jerry Liu's article: https://medium.com/llamaindex-blog/data-agents-eed797d7972f
# 3. Henrik Kniberg youtube video: https://www.youtube.com/watch?v=i-oHvHejdsc&t=616s